# Handwritten Equation Solver
In this project we came up with multiple methods to solve handwritten equations and provide a live demo of the process.

### Project Repo 
https://github.com/opencampus-sh/project_template_folder

## Participants:

- Jeremy von Winckelmann
- Fynn Junge
- Tim Staubert

### Course and Semester
Deep Leraning WS 2022/2023

In [23]:
import torch
from PIL import Image
import numpy as np
from torch import nn as nn

In [45]:
def predict(two_d_array):
    four_d_array = test = np.expand_dims(two_d_array,axis=(0,1))
    clf.eval()
    output = clf(torch.Tensor(test))
    _, predicted = torch.max(output.data, 1)
    classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "-"]
    return classes[predicted]   

In [46]:
predict(np.asarray(img))

'-'